### Imports

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, classification_report
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Get data

In [2]:
train_df = pd.read_csv('C:/Users/Samir/Documents/GitHub/Projet-STT-3795/src/data/wav_files_clean/train_preprocessed_data')
test_df = pd.read_csv('C:/Users/Samir/Documents/GitHub/Projet-STT-3795/src/data/wav_files_clean/test_preprocessed_data')
validation_df = pd.read_csv('C:/Users/Samir/Documents/GitHub/Projet-STT-3795/src/data/wav_files_clean/validation_preprocessed_data')

df = pd.concat([train_df, test_df, validation_df]).reset_index().iloc[:, 2:]
df.rename(columns={'0': 'label'}, inplace=True)

df_without_label = df.iloc[:, 0:-1]

df_without_label

,MFCC_mean_1,MFCC_mean_2,MFCC_mean_3,MFCC_mean_4,MFCC_mean_5,MFCC_mean_6,MFCC_mean_7,MFCC_mean_8,MFCC_mean_9,MFCC_mean_10,...,median_energy,min_energy,max_energy,energy_range,q25_energy,q75_energy,energy_iqr,energy_variability,zero_crossing_rate,low_energy_frame_rate
0,2.157688e-07,-1.192093e-09,-4.827976e-08,1.072884e-08,-4.798174e-08,-1.585484e-07,-1.907349e-08,8.940696e-10,-2.145767e-08,1.192093e-08,...,0.037345,2.265041e-04,0.079646,0.079419,0.014605,0.052177,0.037572,0.724594,0.340000,0.280000
1,2.880891e-07,-4.967054e-08,-3.973643e-08,-1.241763e-09,0.000000e+00,-3.973643e-08,-7.202228e-08,-3.973643e-08,0.000000e+00,1.490116e-08,...,0.060775,4.608796e-03,0.133238,0.128629,0.041807,0.078045,0.036238,0.488379,0.291667,0.208333
2,2.037395e-07,4.118139e-08,2.844767e-08,-6.502325e-09,-2.384186e-08,7.586046e-08,-2.004884e-08,1.950697e-08,3.251163e-08,-1.408837e-08,...,0.021503,4.646347e-04,0.114373,0.113909,0.011146,0.036039,0.024893,0.586159,0.363636,0.327273
3,-1.739811e-07,7.732494e-08,-1.822372e-08,8.054681e-09,-9.665618e-09,1.288749e-08,-2.094217e-08,4.832809e-09,-2.416405e-08,-8.860150e-09,...,0.033641,2.054304e-05,0.095782,0.095761,0.021047,0.060261,0.039214,0.460019,0.324324,0.216216
4,2.781550e-07,-1.655685e-08,3.311369e-08,9.106265e-09,2.317958e-08,2.649095e-08,-1.324548e-08,-1.986822e-08,2.649095e-08,-1.986822e-08,...,0.011589,1.277450e-04,0.047214,0.047086,0.003591,0.019345,0.015754,0.397569,0.291667,0.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33834,-8.316928e-09,-1.478565e-08,1.478565e-08,-7.392824e-09,1.478565e-08,-6.468721e-09,1.478565e-08,-1.293744e-08,5.544618e-09,5.544618e-09,...,0.010918,6.751887e-18,0.108211,0.108211,0.002563,0.026813,0.024251,1.196073,0.286822,0.472868
33835,-3.833195e-07,7.433453e-08,1.335966e-08,1.712777e-09,1.019102e-08,1.096177e-08,8.015797e-08,-7.193664e-08,-1.164688e-08,3.691035e-08,...,0.010180,9.358834e-07,0.091771,0.091770,0.002963,0.023323,0.020360,0.700669,0.287356,0.436782
33836,-7.663454e-08,-2.554485e-08,8.514950e-09,-1.702990e-08,1.277242e-08,8.514950e-09,-1.277242e-08,-1.490116e-08,1.064369e-09,-8.514950e-09,...,0.004112,0.000000e+00,0.216372,0.216372,0.000022,0.074126,0.074104,0.939229,0.232143,0.642857
33837,-5.337048e-07,5.352254e-08,2.729345e-08,5.778001e-09,-3.649264e-09,6.082107e-09,1.338063e-08,9.427265e-09,-3.284337e-08,1.642169e-08,...,0.005400,1.110373e-05,0.049721,0.049709,0.001521,0.013512,0.011991,0.413974,0.306122,0.479592


### Principal components

In [3]:
def get_PCs(dataframe, percentage_variance):
    print()
    scaler = StandardScaler()
    scaled_df = scaler.fit_transform(dataframe)
    print(f'Scaled_df Mean = {np.mean(scaled_df)},\nScaled_df Std = {np.std(scaled_df)}')


    pca_T = PCA()
    pca_T.fit_transform(scaled_df)
    ev = pca_T.explained_variance_
    print()
    print(f'Total variance = {sum(ev)}')

    pca = PCA(percentage_variance/100)
    principal_components = pca.fit_transform(scaled_df)
    explained_variance = pca.explained_variance_
    percentage = sum(pca.explained_variance_ratio_)
    print(f'Real percentage = {percentage}')
    print(f'Variance for {round(percentage*100, 2)}% = {sum(explained_variance)}')
    print(f'Number of PCs for {round(percentage*100, 2)}% = {len(explained_variance)}')
    print(f'Attribute lost = {len(scaled_df[0]) - len(explained_variance)}')
    names = pca.get_feature_names_out()
    return pd.DataFrame(data=principal_components, columns=names)


In [4]:
df_pca = get_PCs(df_without_label, 99)
df_pca['label'] = df['label']



Scaled_df Mean = -2.584758141888949e-10,
Scaled_df Std = 0.9900823995480058

Total variance = 149.00440333353043
Real percentage = 0.9906802950257508
Variance for 99.07% = 147.61572625459794
Number of PCs for 99.07% = 112
Attribute lost = 40


## Models

### Training

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(df_pca.iloc[:, 0:-1], df['label'], stratify=df['label'], test_size=0.1, random_state=42) 

# Model initialization
svm = SVC(verbose=3)
random_forest = RandomForestClassifier(verbose=3)
nn = MLPClassifier(verbose=3)

# Models fiting
print("Training SVM")
svm.fit(X_train, Y_train)
print("Training RFC")
random_forest.fit(X_train, Y_train)


Training SVM
[LibSVM]Training RFC
building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100


[Parallel(n_jobs=1)]: Done  31 tasks      | elapsed:   59.9s


building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73

RandomForestClassifier(verbose=3)

### Predictions

In [15]:
# Models prediction
svm_predictions = svm.predict(X_test)
random_forest_predictions = svm.predict(X_test)

### Metrics

#### Accuracy

In [16]:
svm_accuracy = accuracy_score(Y_test, svm_predictions)
random_forest_accuracy = accuracy_score(Y_test, random_forest_predictions)

print(f'SVM accuracy: {svm_accuracy}')
print(f'RDF accuracy: {random_forest_accuracy}')

SVM accuracy: 0.23670212765957446
RDF accuracy: 0.23670212765957446


#### Confusion matrix

In [17]:
svm_conf_matrix = confusion_matrix(Y_test, svm_predictions)
rdf_conf_matrix = confusion_matrix(Y_test, random_forest_predictions)

#svm_cf_display = ConfusionMatrixDisplay(confusion_matrix=svm_conf_matrix, display_labels=[False, True])
#rdf_cf_display = ConfusionMatrixDisplay(confusion_matrix=rdf_conf_matrix, display_labels=[False, True])

print(svm_conf_matrix)
print()
print(rdf_conf_matrix)

[[10  1 13 ...  1  0  0]
 [ 1 14  0 ...  1  1  0]
 [ 2  0 56 ...  3  0  0]
 ...
 [ 3  3  4 ...  4  2  0]
 [ 0  1  4 ...  1  4  0]
 [ 1  1  1 ...  0  0  5]]

[[10  1 13 ...  1  0  0]
 [ 1 14  0 ...  1  1  0]
 [ 2  0 56 ...  3  0  0]
 ...
 [ 3  3  4 ...  4  2  0]
 [ 0  1  4 ...  1  4  0]
 [ 1  1  1 ...  0  0  5]]


#### Classification report

In [19]:
target = df['label']
report = classification_report(Y_test, svm_predictions)
print(report)

                   precision    recall  f1-score   support

           Arabic       0.11      0.11      0.11        91
           Basque       0.22      0.22      0.22        63
           Breton       0.24      0.48      0.32       117
          Catalan       0.22      0.15      0.18        61
    Chinese_China       0.11      0.05      0.07        60
 Chinese_Hongkong       0.44      0.33      0.38        67
   Chinese_Taiwan       0.19      0.33      0.24       100
          Chuvash       0.23      0.31      0.26        72
            Czech       0.12      0.09      0.10        81
          Dhivehi       0.28      0.29      0.29        65
            Dutch       0.13      0.18      0.15        85
          English       0.08      0.03      0.05        58
        Esperanto       0.11      0.08      0.09        64
         Estonian       0.23      0.19      0.21        54
           French       0.18      0.13      0.15        61
          Frisian       0.11      0.06      0.08       